In [ ]:
import numpy as np
import pandas as pd

Baca Review Produk Lazada

In [ ]:
revlazada = pd.read_csv("/content/drive/MyDrive/LSTMSiamese/data/20191002-reviews.csv")

In [ ]:
len(revlazada)

203787

Mengambil Data yang Konten Review nya tidak **Null**

In [ ]:
revnotnull=revlazada.loc[pd.notnull(revlazada['reviewContent'])]

In [ ]:
len(revnotnull)

107029

Mengambil Data yang Konten Review nya tidak **Duplicate**

In [ ]:
revnotnull = revnotnull.drop_duplicates('reviewContent')

In [ ]:
len(revnotnull)

38071

Mengambil Data yang Konten Review nya memiliki **jumlah kata** lebih dari 10 kata

In [ ]:
rev=[]
revmin = pd.DataFrame()

In [ ]:
revnotnull['lenRev']=[len(i.split()) for i in revnotnull['reviewContent']]

In [ ]:
revnotnull.head(100)

,itemId,category,name,rating,originalRating,reviewTitle,reviewContent,likeCount,upVotes,downVotes,helpful,relevanceScore,boughtDate,clientType,retrievedDate,lenRev
13,100004505,beli-harddisk-eksternal,Andry M.,1,NaN,Pesanan tidak tepat waktu,Pengirim barang tidak sesuai janji. Katanya ex...,1,1,0,True,31.51,18 Mei 2018,mobile,2019-10-02,20
15,100004505,beli-harddisk-eksternal,Lazada Guest,1,NaN,Cara mendapatkan garansi,Bagaimana tv LED saya merek Sharp 24 hitam LC ...,14,14,0,True,22.67,NaN,mobile-app,2019-10-02,38
30,100007498,beli-harddisk-eksternal,Amarullah,5,NaN,Good job,Barang diterima dengan selamat.. Produk sesuai...,0,0,0,True,26.98,11 Nov 2016,mobile,2019-10-02,21
32,100010722,beli-harddisk-eksternal,Dwi,5,NaN,TOP,"Senang belanja disini,pelayanan oke,barang ses...",1,1,0,True,30.51,13 Nov 2017,androidApp,2019-10-02,21
42,100010745,beli-harddisk-eksternal,Anggun W.,5,NaN,Lima Bintang,"Barang Ok, kualitas bagus dengan harga yg cuku...",3,3,0,True,30.75,25 Nov 2016,mobile-app,2019-10-02,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,100363400,beli-harddisk-eksternal,Moch.Mughni,4,NaN,hmmmm,eliminasi 2 hari packing rapi tapi warna gak s...,0,0,0,True,22.49,23 Jul 2018,androidApp,2019-10-02,16
588,100363874,beli-harddisk-eksternal,Ahmad S.,5,NaN,Barang sesuai dengan gambar,"Paket sudah saya terima, terlambat sedikit pen...",0,0,0,True,19.94,30 Apr 2017,mobile-app,2019-10-02,13
598,10041426,beli-harddisk-eksternal,Mia,5,NaN,Good,"Flashdisk baru, waktu pas beli lagi diskon, pa...",0,0,0,True,20.51,30 Okt 2016,mobile,2019-10-02,14
617,100570199,beli-harddisk-eksternal,Muhammad M.,3,NaN,Lama pengiriman dan sistemnya kosong,"8 hari baru sampai, sistemnya kosong root saja...",2,2,0,True,30.34,06 Feb 2017,mobile-app,2019-10-02,17


In [ ]:
revnotnull = revnotnull.loc[revnotnull['lenRev']>10]

In [ ]:
len(revnotnull)

16437

In [ ]:
min(revnotnull['lenRev'])

11

Membersihkan Data : Menghapus simbol, kata repetitif

In [ ]:
revnotnull['reviewClean'] = revnotnull['reviewContent'].str.lower()
revnotnull['reviewClean'] = revnotnull['reviewClean'].str.replace(r"(.)\1{2,}", r"\1") # replace 3 repeated character in words into 1 character
revnotnull['reviewClean'] = revnotnull['reviewClean'].str.replace(r"(^| ).(( ).)*( |$)"," ") # remove 1 standalone character
revnotnull['reviewClean'] = revnotnull['reviewClean'].str.replace(r"\s\s+"," ") # remove multi space after regex

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
revnotnull

,itemId,category,name,rating,originalRating,reviewTitle,reviewContent,likeCount,upVotes,downVotes,helpful,relevanceScore,boughtDate,clientType,retrievedDate,lenRev,reviewClean
13,100004505,beli-harddisk-eksternal,Andry M.,1,NaN,Pesanan tidak tepat waktu,Pengirim barang tidak sesuai janji. Katanya ex...,1,1,0,True,31.51,18 Mei 2018,mobile,2019-10-02,20,pengirim barang tidak sesuai janji. katanya ex...
15,100004505,beli-harddisk-eksternal,Lazada Guest,1,NaN,Cara mendapatkan garansi,Bagaimana tv LED saya merek Sharp 24 hitam LC ...,14,14,0,True,22.67,NaN,mobile-app,2019-10-02,38,bagaimana tv led saya merek sharp 24 hitam lc ...
30,100007498,beli-harddisk-eksternal,Amarullah,5,NaN,Good job,Barang diterima dengan selamat.. Produk sesuai...,0,0,0,True,26.98,11 Nov 2016,mobile,2019-10-02,21,barang diterima dengan selamat.. produk sesuai...
32,100010722,beli-harddisk-eksternal,Dwi,5,NaN,TOP,"Senang belanja disini,pelayanan oke,barang ses...",1,1,0,True,30.51,13 Nov 2017,androidApp,2019-10-02,21,"senang belanja disini,pelayanan oke,barang ses..."
42,100010745,beli-harddisk-eksternal,Anggun W.,5,NaN,Lima Bintang,"Barang Ok, kualitas bagus dengan harga yg cuku...",3,3,0,True,30.75,25 Nov 2016,mobile-app,2019-10-02,21,"barang ok, kualitas bagus dengan harga yg cuku..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82221,9973500,beli-harddisk-eksternal,Lazada Guest,3,NaN,Puas,"Pengiriman lebih cepat , packing rapih Dan ba...",0,0,0,True,9.87,NaN,mobile,2019-10-02,13,pengiriman lebih cepat packing rapih dan baran...
87955,687840070,beli-laptop,Lazada Customer,1,NaN,NaN,I can't login to activated this netbook becaus...,0,0,0,True,62.43,18 Sep 2019,iosApp,2019-10-02,17,can't login to activated this netbook because...
103881,377474276,beli-smart-tv,Yunda A.,4,NaN,NaN,"barang ok, pengiriman super lambat dn trpaksa ...",0,0,0,True,31.01,13 Nov 2018,androidApp,2019-10-02,28,"barang ok, pengiriman super lambat dn trpaksa ..."
195817,377474276,shop-televisi-digital,danang C.,5,NaN,NaN,"mantapp,, terima kasih lazadaaa,, tv coocaa 40...",0,0,0,True,31.01,11 Nov 2018,androidApp,2019-10-02,15,"mantapp,, terima kasih lazada,, tv coocaa 40"" ..."


Tokenisasi

In [ ]:
import spacy
from spacy.lang.id import Indonesian

nlp = Indonesian()

In [ ]:
def df_tokenizer(tweets_df):
    tokenized = [token.text for token in nlp(tweets_df)]
    return tokenized

In [ ]:
tokenized = revnotnull['reviewClean'].apply(df_tokenizer)

Handle kata slang

In [ ]:
import json
def slang(tokenized_):
    slang_word = json.loads(open('/content/drive/MyDrive/LSTMSiamese/data/slang.json','r').read())
    for index in range(len(tokenized_)):
        for key, value in slang_word.items():
            for v in value:
                if tokenized_[index] == v:
                    tokenized_[index] = key
                else:
                    continue
    value = " ".join(str(v) for v in tokenized_) #join as string
    return value

In [ ]:
slang_cleaned = tokenized.apply(slang)

In [ ]:
slang_cleaned.tail()

82221     pengiriman lebih cepat packing rapih dan baran...
87955       can't login to activated this netbook karena...
103881    barang oke , pengiriman super lambat dn trpaks...
195817    mantapp , , terima kasih lazada , , tv coocaa ...
199781    wow keren habis , , led tv kualitas dewa , pak...
Name: reviewClean, dtype: object

In [ ]:
#slang_cleaned = slang_cleaned.str.replace(r"(^| ).(( ).)*( |$)"," ") # remove 1 standalone character

In [ ]:
slang_cleaned.tail()

82221     pengiriman lebih cepat packing rapih dan baran...
87955       can't login to activated this netbook karena...
103881    barang oke , pengiriman super lambat dn trpaks...
195817    mantapp , , terima kasih lazada , , tv coocaa ...
199781    wow keren habis , , led tv kualitas dewa , pak...
Name: reviewClean, dtype: object

In [ ]:
revnotnull['reviewClean'] = slang_cleaned

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
len(revnotnull['reviewClean'])

16437

Menyimpan Data yang Memiliki Review ***dengan panjang lebih dari 30 kata***

In [ ]:
revnotnull.loc[revnotnull['lenRev']>30].to_csv("/content/drive/MyDrive/LSTMSiamese/data/revLazCleanMin30words.csv")

In [ ]:
tes = pd.read_csv("/content/drive/MyDrive/LSTMSiamese/data/revLazCleanMin30words.csv")

In [ ]:
len(tes)

2844